# Домашняя работа 1

In [1]:
import newspaper
from newspaper import Article
import pandas as pd

In [4]:
transport = 'http://tass.ru/transport/'
economy = 'http://tass.ru/makroekonomika/'
avtosport = 'http://tass.ru/avtosport/'
politics = 'http://tass.ru/politika/'

In [5]:
def get_topic(url):
    texts = []
    for i in range(1000000, 9999999):
        if len(texts) < 4:
            article = Article(url + str(i))
            try:
                article.download()
                article.parse()
                texts.append(article.text)
            except:
                pass
    return texts

In [ ]:
t_text = get_topic(transport)
e_text = get_topic(economy)
a_text = get_topic(avtosport)
p_text = get_topic(politics)

In [7]:
texts = pd.DataFrame()
texts['transport'] = t_text
texts['economy'] = e_text
texts['avtosport'] = a_text
texts['politics'] = p_text
texts.to_csv('texts.csv', index=False)

In [9]:
texts = pd.read_csv('texts.csv')

In [10]:
texts.shape

(200, 4)

In [11]:
import os
import sys
import json
import re

from pymystem3 import Mystem
m = Mystem()

In [12]:
def clean(x):
    res = []
    for line in x:
        curr = []
        for word in line:
            #print(word)
            if (' ' not in word) and ('\n' not in word):
                curr.append(word)
        res.append(curr)
    return res

In [13]:
def get_lems(data):
    data = data.apply(lambda x: x.replace('?', '.').replace('!', '.'))
    data = data.apply(lambda x: x.split('.'))
    data = data.apply(lambda x: [re.sub(r'[^\w\s]', ' ', i.lower()) for i in x])
    data = data.apply(lambda x: [re.sub('\d', ' ', i) for i in x])
    data = data.apply(lambda x: [re.sub(r'[\n\r\t]', ' ', i)for i in x])
    data = data.apply(lambda x: [' '.join(re.split("[^а-я]*", i.lower()))for i in x])
    data = data.apply(lambda x: [m.lemmatize(i) for i in x])
    data = data.apply(lambda x: clean(x))
    return data

In [14]:
texts.head()

,politics,science,culture,economy
0,"('Встреча Лаврова и Зарифа прорабатывается, за...",('SpaceX: при запуске спутника Zuma ракета Fal...,('Партнерша Ведерникова по ГАБТ назвала его че...,('Цена на нефть марки Brent впервые с мая 2015...
1,('Путин обсудил с Силуановым и Пучковым госпро...,('Генетики раскрыли родословную первых викинго...,('Солистка Большого театра Татьяна Ерастова на...,('Цена на нефть марки WTI впервые с 2014 года ...
2,('Медведев назначил Шпади замглавы Росжелдора'...,('Ученые нашли в Австралии птиц с крыльями тем...,('Солист Большого театра Владимир Маторин отме...,('Натуральное свинство: чем грозит России иск ...
3,('Медведев освободил Луковникова от должности ...,('В Финляндии разработают спрей для лечения иг...,('Франшизы и супергерои: самые ожидаемые фильм...,('Россия попросит ВТО созвать панель арбитров ...
4,('ЦИК получил документы для регистрации Грудин...,"('РКК ""Энергия"" предложила заряжать лунные ста...","('Умер оперный певец Александр Ведерников', ' ...",('Тень независимости: почему Украина не соскоч...


In [15]:
texts.science = get_lems(texts.science)
texts.economy = get_lems(texts.economy)
texts.culture = get_lems(texts.culture)
texts.politics = get_lems(texts.politics)
texts.head()

/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


,politics,science,culture,economy
0,"[[встреча, лавров, и, зарифа, прорабатываться,...","[[при, запуск, спутник, ракета, работать, штат...","[[партнерша, ведерникова, по, габт, называть, ...","[[цена, на, нефть, марка, впервые, с, май, год..."
1,"[[путин, обсуждать, с, силуанов, и, пучков, го...","[[генетик, раскрывать, родословная, первый, ви...","[[солистка, большой, театр, татьяна, ерастова,...","[[цена, на, нефть, марка, впервые, с, год, пре..."
2,"[[медведев, назначать, шпадь, замглавы, росжел...","[[ученый, находить, в, австралия, птица, с, кр...","[[солист, большой, театр, владимир, маторина, ...","[[натуральный, свинство, что, грозить, россия,..."
3,"[[медведев, освобождать, луковников, от, должн...","[[в, финляндия, разрабатывать, спрей, для, леч...","[[франшиза, и, супергерой, самый, ожидать, фил...","[[россия, попросить, вто, созывать, панель, ар..."
4,"[[цик, получать, документ, для, регистрация, г...","[[ркк, энергия, предлагать, заряжать, лунный, ...","[[умирать, оперный, певец, александр, ведерник...","[[тень, независимость, почему, украина, не, со..."


In [16]:
uni = []
for col in texts.columns:
    print(col)
    for t in texts[col]:
        for i in t:
            uni += i

uni = sorted(list(set(uni)))

politics
science
culture
economy


In [17]:
print(uni[:10])

['а', 'аа', 'аарон', 'абакумов', 'аббас', 'аббревиатура', 'абдалла', 'абдель', 'абдельхафиз', 'абдул']


In [18]:
num2v = {}
v2num = {}
for i, v in enumerate(uni):
    v2num[v] = i
    num2v[i] = v
v2num

{'а': 0,
 'аа': 1,
 'аарон': 2,
 'абакумов': 3,
 'аббас': 4,
 'аббревиатура': 5,
 'абдалла': 6,
 'абдель': 7,
 'абдельхафиз': 8,
 'абдул': 9,
 'абердинский': 10,
 'абиогенный': 11,
 'аблаев': 12,
 'абонент': 13,
 'абориген': 14,
 'абрам': 15,
 'абрамов': 16,
 'абрамович': 17,
 'абсолютно': 18,
 'абсолютный': 19,
 'абстрактный': 20,
 'абсурд': 21,
 'абсурдность': 22,
 'абсурдный': 23,
 'абхазия': 24,
 'аварийный': 25,
 'авария': 26,
 'август': 27,
 'авербух': 28,
 'авиабаза': 29,
 'авиазавод': 30,
 'авиаперевозка': 31,
 'авиасообщение': 32,
 'авиастроительный': 33,
 'авиатехник': 34,
 'авиационный': 35,
 'авива': 36,
 'авилов': 37,
 'аврора': 38,
 'австралийский': 39,
 'австралия': 40,
 'австрийский': 41,
 'австрия': 42,
 'автобус': 43,
 'автоваз': 44,
 'автограф': 45,
 'автомат': 46,
 'автоматизированный': 47,
 'автоматизм': 48,
 'автоматика': 49,
 'автоматически': 50,
 'автоматический': 51,
 'автомашина': 52,
 'автомобиль': 53,
 'автомобильный': 54,
 'автономия': 55,
 'автономный': 56

In [19]:
json.dump(num2v, open('num2v.json', 'w'))
json.dump(v2num, open('v2num.json', 'w'))

In [20]:
def get_nums(x):
    res = []
    for line in x:
        res.append(' '.join(str(v2num[i]) for i in line))
    return res

In [21]:
for col in texts.columns:
    texts[col] = texts[col].apply(lambda x: get_nums(x))
texts.head()

,politics,science,culture,economy
0,[1744 5287 3811 3601 9675 3687 1098 5971 10583...,[9314 3580 11695 10102 9911 13834 4795 12954 1...,[8122 1195 8502 1977 6459 7546 13591 10849 140...,[13500 6381 7006 5707 1655 10684 5605 2219 917...
1,[9865 7334 10684 11056 3811 9868 2326 1600 925...,[2081 10133 10523 8191 1362 11129 12993 10240 ...,[11465 953 12172 12145 3199 6459 1194 10725 65...,[13500 6381 7006 5707 1655 10684 2219 9172 333...
2,[5822 6457 13822 3499 10566 9228 9133 10649 28...,[12993 6636 1098 40 9827 10684 5165 12216 7151...,[11464 953 12172 1427 5774 7862 1405 1194 1098...,[6629 10862 13685 2413 10583 4139 3148 1098 17...
3,[5822 7695 5535 7770 2952 3499 10566 9228 9133...,[1098 13161 10063 11685 2862 5400 3835 1085 64...,[13249 3811 11982 10734 7458 13143 2219 2232 7...,[10583 8935 1755 11443 8062 395 8502 11666 106...
4,[13527 8862 2933 2862 10307 2420 4337 1098 925...,[10489 14002 9190 3611 5540 11769 5292 10684 7...,[12813 7578 8157 182 1194 11386 3811 10580 757...,[12226 6757 9094 12767 6669 11537 10684 1994 5...


In [85]:
def to_xml(df, filename=None):
    xml = []
    for col in df.columns:
        i = 0
        xml = ['<' + col + '>']
        for text in df[col]:
            xml.append(' <' + str(i) + '>')
            j = 0
            for line in text:
                xml.append('  <row' + str(j) + '>' + line + '</row' + str(j) + '>')
                j += 1
            xml.append('</' + str(i) + '>')
            i += 1
        xml.append('</' + col + '>')
    xml = '\n'.join(xml)
    if filename is None:
        return xml
    with open(filename, 'w') as f:
        f.write(xml)

In [86]:
to_xml(texts, 'texts.xml')

In [22]:
texts.shape[0]

200

# Классная работа 2

In [28]:
data =  pd.read_csv('texts.csv')

In [29]:
v2text = {}
for i_col, col in enumerate(texts.columns):
    for i_text, text in enumerate(texts[col]):
        for line in text:
            for word in line.split():
                word = int(word)
                if word not in v2text:
                    v2text[word] = [i_col * texts.shape[0] + i_text]
                else:
                    if i_col * texts.shape[0] + i_text not in v2text[word]:
                        v2text[word].append(i_col * texts.shape[0] + i_text)
print(v2text[123])

[84]


In [30]:
def get_lems_query(data):
    data = re.sub(r'[^\w\s]', ' ', data.lower())
    data = re.sub('\d', ' ', data)
    data = re.sub(r'[\n\r\t]', ' ', data)
    data = ' '.join(re.split("[^а-я]*", data))
    data = m.lemmatize(data)
    data = [i for i in data if ' ' not in i and '\n' not in i]
    return list(set(data))

In [31]:
querry = 'россия сша война'
querry = get_lems_query(querry)
querry

/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


['война', 'сша', 'россия']

In [32]:
from itertools import combinations
from collections import Counter

def solve():
    for n in range(len(querry), 0, -1):
        for comb in combinations(querry, n):
            res = []
            for word in comb:
                if word in v2num:
                    res += v2text[v2num[word]]
            res = Counter(res)
            res = [(l,k) for k,l in sorted([(j,i) for i,j in res.items()], reverse=True)]
            if res[0][1] < n:
                break
            print(comb)
            return [i[0] for i in res if i[1] == res[0][1]]

In [33]:
res = solve()
for i in res:
    i_col = i // texts.shape[0]
    i_text = i % texts.shape[0]
    print(data[data.columns[i_col]].iloc[i_text])
    print ('==================================================================')

('война', 'сша', 'россия')
('Глава ВТБ сравнил новые санкции США с объявлением войны', '  Глава ВТБ Андрей Костин предупредил о "растущей угрозе военного конфликта" в\xa0Европе и\xa0сравнил возможное введение США новых экономических санкций против России с "объявлением войны", пишет "Файненшл таймс". \n "Мы на\xa0пороге новой гонки вооружений. НАТО требует больше оружия и\xa0размещает все больше оружия в\xa0Европе. Россия ответит тем же. Кто от\xa0этого выиграет? Только генералы и\xa0производители оружия. Америка считает, что Европа должна заплатить за\xa0это больше. Кому это надо? Это очень опасно",\xa0— сказал Костин в\xa0интервью газете на\xa0полях Всемирного экономического форума в\xa0Давосе. По словам Костина, любые экономические санкции против российских организаций только обострят ситуацию и\xa0сродни "объявлению войны". \n \n "На мой взгляд, это очень опасная ситуация, хуже "холодной войны". Считаю, что американцы играют с\xa0огнем, потому что наши отношения продолжают ухудшать

In [34]:
len(res)

5

#  Домашнее задание 2

In [7]:
import pandas as pd
import re

from pymystem3 import Mystem
m = Mystem()

In [8]:
data =  pd.read_csv('texts.csv')

In [9]:
def get_lems_dz2(data):
    data = re.sub(r'[^\w\s]', ' ', data.lower())
    data = re.sub('\d', ' ', data)
    data = re.sub(r'[\n\r\t]', ' ', data)
    data = ' '.join(re.split("[^а-я]*", data))
    data = m.lemmatize(data)
    data = [i for i in data if ' ' not in i and '\n' not in i]
    return ' '.join(data)

In [10]:
data.head()

,politics,science,culture,economy
0,"('Встреча Лаврова и Зарифа прорабатывается, за...",('SpaceX: при запуске спутника Zuma ракета Fal...,('Партнерша Ведерникова по ГАБТ назвала его че...,('Цена на нефть марки Brent впервые с мая 2015...
1,('Путин обсудил с Силуановым и Пучковым госпро...,('Генетики раскрыли родословную первых викинго...,('Солистка Большого театра Татьяна Ерастова на...,('Цена на нефть марки WTI впервые с 2014 года ...
2,('Медведев назначил Шпади замглавы Росжелдора'...,('Ученые нашли в Австралии птиц с крыльями тем...,('Солист Большого театра Владимир Маторин отме...,('Натуральное свинство: чем грозит России иск ...
3,('Медведев освободил Луковникова от должности ...,('В Финляндии разработают спрей для лечения иг...,('Франшизы и супергерои: самые ожидаемые фильм...,('Россия попросит ВТО созвать панель арбитров ...
4,('ЦИК получил документы для регистрации Грудин...,"('РКК ""Энергия"" предложила заряжать лунные ста...","('Умер оперный певец Александр Ведерников', ' ...",('Тень независимости: почему Украина не соскоч...


In [11]:
dz2 = []
for col in data.columns:
    for line in data[col]:
        dz2.append([col, line])
dz2 = pd.DataFrame(dz2)
dz2.columns = ['label', 'text']
dz2.text = dz2.text.apply(lambda x: get_lems_dz2(x))
dz2.head()

/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


,label,text
0,politics,встреча лавров и зарифа прорабатываться заявля...
1,politics,путин обсуждать с силуанов и пучков госпрограм...
2,politics,медведев назначать шпадь замглавы росжелдор пр...
3,politics,медведев освобождать луковников от должность з...
4,politics,цик получать документ для регистрация грудинин...


In [12]:
dz2.iloc[690:695]

,label,text
690,economy,ренов планировать наращивать инвестиция вдвое ...
691,economy,набиуллина рассказывать об изменение зависимос...
692,economy,период выплата налог не являться пиковый заявл...
693,economy,биткоин торговаться низко отметка в десять тыс...
694,economy,защита от риск в россия открывать банк для опе...


In [13]:
dz2.iloc[90:95]

,label,text
90,politics,в штаб путин рассказывать как волонтер мочь пр...
91,politics,зарабатывать официальный сайт избирательный шт...
92,politics,в москва зарабатывать общественный приемный пр...
93,politics,в штаб путин рассказывать во сколько обходитьс...
94,politics,эксперт оценивать интерес к выборы президент р...


In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(dz2.text)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(800, 14174)

In [15]:
X_train_tfidf.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
request = ['россия сша война']
request_counts = count_vect.transform(request)
request_tfidf = tfidf_transformer.transform(request_counts)
request_tfidf.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim = []
for i, vec in enumerate(X_train_tfidf):
    cos_sim.append([i, cosine_similarity(vec, request_tfidf)])

In [18]:
cos_sim = [(l,k) for k,l in sorted([(j,i) for i,j in cos_sim], reverse=True)]
cos_sim = [i[0] for i in cos_sim if i[1] == cos_sim[0][1]]

In [19]:
for i in cos_sim:
    i_col = i // data.shape[0]
    i_text = i % data.shape[0]
    print(data[data.columns[i_col]].iloc[i_text])
    print ('==================================================================')

('Глава ВТБ сравнил новые санкции США с объявлением войны', '  Глава ВТБ Андрей Костин предупредил о "растущей угрозе военного конфликта" в\xa0Европе и\xa0сравнил возможное введение США новых экономических санкций против России с "объявлением войны", пишет "Файненшл таймс". \n "Мы на\xa0пороге новой гонки вооружений. НАТО требует больше оружия и\xa0размещает все больше оружия в\xa0Европе. Россия ответит тем же. Кто от\xa0этого выиграет? Только генералы и\xa0производители оружия. Америка считает, что Европа должна заплатить за\xa0это больше. Кому это надо? Это очень опасно",\xa0— сказал Костин в\xa0интервью газете на\xa0полях Всемирного экономического форума в\xa0Давосе. По словам Костина, любые экономические санкции против российских организаций только обострят ситуацию и\xa0сродни "объявлению войны". \n \n "На мой взгляд, это очень опасная ситуация, хуже "холодной войны". Считаю, что американцы играют с\xa0огнем, потому что наши отношения продолжают ухудшаться, и\xa0не мы в\xa0этом ви